<a href="https://www.kaggle.com/code/sophiazmliang/music-recommendation-system?scriptVersionId=190616125" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **Music Recommendation System**

## **Problem Definition**

### **The Context:**

  Online streaming platforms like Spotify offer millions of songs in their music library. Users often feel overwhelmed by the sheer volume of choices. Developing a recommendation system that suggests relevant songs based on users' historical interactions can significantly enhance user engagement. This increased engagement encourages users to spend more time on the platform, leading to higher user satisfaction and increased subscription renewals or higher advertising revenue from free-tier users, thus improving the platform's overall revenue.

  Moreover, such recommendation systems can aid in discovering emerging music trends, guiding strategic decisions on content acquisition, partnerships, and marketing. Continuous improvement and innovation in recommendation algorithms can differentiate a platform from its competitors, establishing leadership in the tech and music streaming industries.
  

### **The objective:**

 The primary goal of this analysis is to build a recommendation system that proposes the top 5 songs for a user based on the likelihood of listening to those songs.

 This system aims to enhance the user experience by providing personalized content that aligns with individual preferences, allowing users to discover new music they might not have found on their own, thereby enriching their overall experience.

  By achieving this goal, the system seeks to increase user engagement, satisfaction, and retention, ultimately driving higher subscription renewals and advertising revenue. Additionally, the system can help management identify emerging music trends and market patterns, enabling strategic decision-making.


### **The key questions:**

- What user/song data is available and how can it be used?
- What algorithms and techniques (e.g., collaborative filtering, content-based filtering, hybrid methods) can be used?
- How can user interactions with songs be quantified and used to train the model?
- How to handle data sparsity?
- How to evaluate the recommendation system? What metrics (e.g., precision, recall, F1 score, RMSE) should be used?
- How to address potential biases in the recommendation algorithm to ensure fairness?
- How can the system help identify emerging music trends for strategic decision-making?

### **The problem formulation**:

- Data Collection and Preprocessing: Gathering data on user interactions, song features, and metadata. Cleaning and preprocessing this data to make it suitable for analysis.
- Modeling: Using machine learning algorithms (e.g., collaborative filtering, content-based filtering, hybrid models) to predict user preferences.
- Evaluation: Measuring the accuracy and effectiveness of the recommendations using metrics like precision, recall, and RMSE.
- Optimization: Tuning the models to improve performance and scalability.


## **Data Dictionary**

The core data is the Taste Profile Subset released by the Echo Nest as part of the Million Song Dataset. There are two files in this dataset. The first file contains the details about the song id, titles, release, artist name, and the year of release. The second file contains the user id, song id, and the play count of users.

**song_data**

- song_id - A unique id given to every song
- title - Title of the song
- Release - Name of the released album
- Artist_name - Name of the artist
- year - Year of release

**count_data**

- user _id - A unique id given to the user
- song_id - A unique id given to the song
- play_count - Number of times the song was played

## **Data Source**
http://millionsongdataset.com/

In [ ]:
# Mounting the drive
from google.colab import drive
drive.mount('/content/drive')

### **Importing Libraries and the Dataset**

In [ ]:
# Used to ignore the warning given as output of the code
import warnings
warnings.filterwarnings('ignore')

# Basic libraries of python for numeric and dataframe computations
import numpy as np
import pandas as pd

# Set the display width to a larger value
pd.set_option('display.width', 200)  # Adjust the value as needed

# Import Matplotlib the Basic library for data visualization
import matplotlib.pyplot as plt

# Import seaborn - Slightly advanced library for data visualization
import seaborn as sns

# Import the required library to compute the cosine similarity between two vectors
from sklearn.metrics.pairwise import cosine_similarity

# Import defaultdict from collections A dictionary output that does not raise a key error
from collections import defaultdict

# Impoort mean_squared_error : a performance metrics in sklearn
from sklearn.metrics import mean_squared_error

### **Load the dataset**

In [ ]:
# Importing the datasets
count_df = pd.read_csv('/content/drive/MyDrive/Colab/Capstone_Project/count_data.csv')
song_df = pd.read_csv('/content/drive/MyDrive/Colab/Capstone_Project/song_data.csv')

### **Understanding the data by viewing a few observations**

In [ ]:
# Display first 10 records of count_df data
count_df.head(10)

In [ ]:
# Display first 10 records of song_df data
song_df.head(10)

### **Let us check the data types and and missing values of each column**

In [ ]:
# Display info of count_df
count_df.info()

In [ ]:
# Non-Null Count is not showed, maybe due to large dataset. Check directly.
count_df.isnull().sum()

In [ ]:
# Display info of song_df
song_df.info()

In [ ]:
# Check the unique values of song_id in song_df
song_df['song_id'].nunique()

In [ ]:
# Check the unique values of user_id in count_df
count_df['user_id'].nunique()

In [ ]:
# Check the unique values of song_id in count_df
count_df['song_id'].nunique()

In [ ]:
# Group by user_id and song_id and count the number of unique pairs in count_df
unique_pairs_alt = count_df.groupby(['user_id', 'song_id']).size().reset_index(name='counts')

# Count the number of unique pairs
len(unique_pairs_alt)

#### **Observations and Insights:**


*   Both datasets are very large. There are 70,000+ users and almost 100,000 songs.
*   Only 10,000 songs are played, we can remove the irrelevant songs to trim down dataset size to 10%.
*   The 'Unnamed: 0' column is no use, can drop it later.
*   'play_count' and 'year' are numeric columns, the others are all string columns.
*   There are some missing values in song_df. Further check it after the big trim.

*   There are some duplicate (song_id) records in song_df. We can further clean it after the big trim.
*   Some song_id records have 0 in 'year' column. Need to minimize it during clean up.
*   No duplicate (user_id + song_id) record in count_df.






### **Trim down the dataset to a more manageable size and do some cleanup**

**Remove the irrelevant songs.**

In [ ]:
# Remove those song_id records which do not appear in count_df
song_df = song_df[song_df['song_id'].isin(count_df['song_id'])]

# Check song_id unique values afterwards
song_df['song_id'].nunique()

In [ ]:
# Check song_df info() again.
song_df.info()

**After trim, the no. of unique song_id in song_df matches with count_df, also there are no missing values.**

**Now let's handle the duplicate records and 0-year issues.**

In [ ]:
# Divide into two datasets, one with non-zero year and one with zero year
song_with_year_df = song_df[song_df.year != 0]
song_without_year_df = song_df[song_df.year == 0]

# Drop duplicate song_id in both sets
song_with_year_df = song_with_year_df.drop_duplicates(subset='song_id')
song_without_year_df = song_without_year_df.drop_duplicates(subset='song_id')

# Remove those zero-year records if their song_id exist in non-zero-year dataset
song_without_year_df = song_without_year_df[~song_without_year_df['song_id'].isin(song_with_year_df['song_id'])]

# Combine the two datasets again and display info()
song_df = pd.concat([song_with_year_df,song_without_year_df],axis=0)
song_df.info()

**The clean up for song_df complete. Now merge it with count_df and drop the column 'Unnamed: 0'.**

In [ ]:
# Left merge count_df and song_df on "song_id". Name the obtained dataframe as "df"
df = pd.merge(count_df, song_df, on = 'song_id', how = 'left')

# Drop the column 'Unnamed: 0'
df = df.drop(['Unnamed: 0'], axis = 1)

In [ ]:
# Display first 5 records of df data
df.head()

**Think About It:** As the user_id and song_id are encrypted. Can they be encoded to numeric features?

**Encode user_id and song_id in df and song_df.**

In [ ]:
# Import LableEncoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Apply label encoding for "user_id" and "song_id"

# Initialize LabelEncoders
user_id_encoder = LabelEncoder()
song_id_encoder = LabelEncoder()

# Fit and transform the 'user_id' column
df['user_id'] = user_id_encoder.fit_transform(df['user_id'])

# Fit and transform the 'song_id' column
df['song_id'] = song_id_encoder.fit_transform(df['song_id'])

# Display first 5 records of df data
df.head()

In [ ]:
# Transform the 'song_id' column in song_df too with the same encoder
song_df['song_id'] = song_id_encoder.transform(song_df['song_id'])

# Display first 5 records of song_df data
song_df.head()

**Think About It:** As the data also contains users who have listened to very few songs and vice versa, is it required to filter the data so that it contains users who have listened to a good count of songs and vice versa?

A dataset of size 2000000 rows x 7 columns can be quite large and may require a lot of computing resources to process. This can lead to long processing times and can make it difficult to train and evaluate your model efficiently.
In order to address this issue, it may be necessary to trim down your dataset to a more manageable size.

**Locate the high-engagement users.**


In [ ]:
# Get the column containing the users
users = df.user_id

# Create a dictionary that maps users(listeners) to the number of songs that they have listened to
playing_count = dict()

for user in users:
    # If we already have the user, just add 1 to their playing count
    if user in playing_count:
        playing_count[user] += 1

    # Otherwise, set their playing count to 1
    else:
        playing_count[user] = 1

In [ ]:
# Show the song count distribution among users
user_song_count = pd.DataFrame(list(playing_count.items()), columns=['user_id', 'song_count'])

plt.figure(figsize=(12, 6))
sns.histplot(data=user_song_count, x='song_count', binwidth=1)
plt.locator_params(axis='x', nbins=15)
plt.xlabel('Song Count', fontsize=10)
plt.ylabel('No. of Users', fontsize=10)
plt.xticks(fontsize=8, rotation=90)
# Set x-ticks at specific intervals
ax = plt.gca()
ax.set_xticks(np.arange(0, 800, 10))
plt.yticks(fontsize=8)
ax.set_yticks(np.arange(0, 4000, 100))
plt.show()

In [ ]:
# Find the cutoff point for users
user_song_count.describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95])

*   **Choose 90 songs as cut off point. They're the top 5% users who demonstrate higher music consumption.**


In [ ]:
# We want our users to have listened at least 90 songs
SONG_COUNT_CUTOFF = 90

# Create a list of users who need to be removed
remove_users = []

for user, num_songs in playing_count.items():

    if num_songs < SONG_COUNT_CUTOFF:
        remove_users.append(user)

df = df.loc[ ~ df.user_id.isin(remove_users)]

**Locate the popular songs.**


In [ ]:
# Get the column containing the songs
songs = df.song_id

# Create a dictionary that maps songs to its number of users(listeners)
playing_count = dict()

for song in songs:
    # If we already have the song, just add 1 to their playing count
    if song in playing_count:
        playing_count[song] += 1

    # Otherwise, set their playing count to 1
    else:
        playing_count[song] = 1

In [ ]:
# Find the cutoff point for songs
song_user_count = pd.DataFrame(list(playing_count.items()), columns=['song_id', 'user_count'])
song_user_count.describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95])

**Choose 120 users as cut off point. It will cover the top 50% songs.**

In [ ]:
# We want our song to be listened by atleast 120 users to be considred
LISTENER_COUNT_CUTOFF = 120

remove_songs = []

for song, num_users in playing_count.items():
    if num_users < LISTENER_COUNT_CUTOFF:
        remove_songs.append(song)

df_final= df.loc[ ~ df.song_id.isin(remove_songs)]

In [ ]:
# Check df_final record numbers
df_final.info()

**As we do not have ratings data of the songs. In this case, we are going to use play_count as a proxy for ratings with the assumption that the more the user listens to a song, the higher the chance that they like the song. We need to check the play_count values distribution.**

In [ ]:
# Find the distribution of play_count values
df_final['play_count'].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95])

Out of all the songs available, songs with play_count less than or equal to 5 are in almost 90% abundance. So for building the recommendation system let us consider only those songs. The outliers, such as users who have listened to a song an excessively high number of times, can skew the analysis.

In [ ]:
# Keep only records of songs with play_count less than or equal to (<=) 5
df_final = df_final[df_final.play_count<=5]

In [ ]:
# Check the shape of the data
df_final.shape

In [ ]:
# Remove those song_id records which do not appear in df_final
song_df = song_df[song_df['song_id'].isin(df_final['song_id'])]

# Check song_df info afterwards
song_df.info()

## **Exploratory Data Analysis**

### **Let's check the total number of unique users, songs, artists in the data**

Total number of unique user id

In [ ]:
# Display total number of unique user_id
df_final['user_id'].nunique()

Total number of unique song id

In [ ]:
# Display total number of unique song_id
df_final['song_id'].nunique()

Total number of unique artists

In [ ]:
# Display total number of unique artists
df_final['artist_name'].nunique()

#### **Observations and Insights:**


**We trim down the dataset to only consider:**

1.   users have listened to at least 90 songs (high-engagement user)
2.   songs have been listened by at least 120 users (high-quality music)
3.   play count less than or equal to 5 (excluding outliers which can skew the analysis)


After the trimming process, the dataset has been condensed to encompass **110,000+ user interactions** involving **560+ songs, 230+ artists and 3000+ users**. This refined dataset provides a more manageable and computationally efficient foundation for constructing recommendation systems.

Based on the count of unique users and songs, there exists a potential for 1,776,265 interactions (3155 users * 563 songs) within the dataset. However, the actual recorded interactions amount to only 117,876, representing approximately **6.6%** of the total potential interactions. This discrepancy underscores the reality that not every user has engaged with every song in the dataset, a common occurrence in such datasets.

While this presents an opportunity to develop recommendation systems for suggesting songs that users have yet to interact with, it also introduces **the challenge of data sparsity**. Addressing this challenge will be paramount in ensuring the efficacy and relevance of the recommendation algorithms.

### **Let's find out about the most interacted songs and interacted users**

Most interacted songs

In [ ]:
# Counting the number of users who have listened to a certain song
song_user_count = pd.DataFrame(df_final['song_id'].value_counts())

# Merge with song_df to show the most interacted songs' info
song_user_count_with_title = pd.merge(song_user_count,song_df, on = 'song_id', how = 'left')
song_user_count_with_title

In [ ]:
# Plotting distributions of play_count for 751 interactions with song_id 8582

# Let us fix the size of the figure
plt.figure(figsize = (7, 7))

df_final[df_final['song_id'] == 8582]['play_count'].value_counts().plot(kind = 'bar')

# This gives a label to the variable on the x-axis
plt.xlabel('Play Count')

# This gives a label to the variable on the y-axis
plt.ylabel('Count')

# This displays the plot
plt.show()

In [ ]:
# Show the user count distribution among songs

plt.figure(figsize=(12, 6))
sns.histplot(data=song_user_count, x='count')
plt.xlabel('User Count', fontsize=10)
plt.ylabel('No. of Songs', fontsize=10)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.show()

In [ ]:
# Plotting distributions of play_count for all interactions with all song_id

# Let us fix the size of the figure
plt.figure(figsize = (7, 7))

df_final['play_count'].value_counts().plot(kind = 'bar')

# This gives a label to the variable on the x-axis
plt.xlabel('Play Count')

# This gives a label to the variable on the y-axis
plt.ylabel('Count')

# This displays the plot
plt.show()

Most interacted users

In [ ]:
# Counting the number of songs which have been listened by a certain user to show the most interacted users
user_song_count = pd.DataFrame(df_final['user_id'].value_counts())
user_song_count

In [ ]:
# Show the song count distribution among users

plt.figure(figsize=(12, 6))
sns.histplot(data=user_song_count, x='count')
plt.xlabel('Song Count', fontsize=10)
plt.ylabel('No. of Users', fontsize=10)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.show()

#### **Observations and Insights:**

*   The most interacted song has 751 users.
*   The most interacted user listened to 243 songs.
*   Among the 110,000+ interactions, the majorities' play count fall between 1 and 2. Over 70000 have play count as 1, Over 20000 have play count as 2, only around 5000 have play count as 5.  
*   No matter songs, users or interaction play counts, they all follow the power law, ie. a small number of items (songs, users, or interaction play counts) account for the majority of the occurrences, while the majority of items have relatively low frequencies. For example, a few popular songs have a lot of interactions, while the majority of songs have a lot fewer.


*   It underscores the complexities associated with data sparsity and the importance of mitigating potential biases inherent in the recommendation algorithm.



Songs released on yearly basis

In [ ]:
# Find out the number of songs released in a year, use the songs_df
  # Hint: Use groupby function on the 'year' column

  # Group by 'year' and count occurrences of 'song_id'
yearly_song_counts = song_df.groupby(by='year')['song_id'].count()

# Convert the Series to a DataFrame and reset the index
yearly_song_counts = yearly_song_counts.reset_index()

# Rename the columns for clarity
yearly_song_counts.columns = ['year', 'count']

# Filter out rows where 'year' is 0
yearly_song_counts = yearly_song_counts[yearly_song_counts['year'] != 0]

# Set the figure size
plt.figure(figsize=(10, 6))

# Create a barplot plot with y label as "number of titles played" and x -axis year
plt.bar(yearly_song_counts['year'], yearly_song_counts['count'], color='skyblue',width=0.8)

# Set the x label of the plot
plt.xlabel('Year')
plt.xticks(yearly_song_counts['year'],rotation=90)  # Ensure all years are shown on x-axis

# Set the y label of the plot
plt.ylabel('Number of Titles Played')

# Show the plot
plt.title('Number of Titles Played Per Year')
plt.show()

Songs' user count by song release year

In [ ]:
# Group by year and count unique user_id
user_count_by_year = df_final.groupby('year')['user_id'].nunique().reset_index()
# Filter out rows where 'year' is 0
user_count_by_year = user_count_by_year[user_count_by_year['year'] != 0]

In [ ]:
# Plot the barplot
plt.figure(figsize=(10, 5))
plt.bar(user_count_by_year['year'], user_count_by_year['user_id'], color='skyblue')
plt.xlabel('Year')
plt.xticks(user_count_by_year['year'],rotation=90)  # Ensure all years are shown on x-axis
plt.ylabel('Count of Unique Users')
plt.title('Count of Unique Users by Year')
plt.xticks(user_count_by_year['year'])
plt.show()

Songs' total play count by song release year

In [ ]:
song_year_play_count = df_final.groupby('year')['play_count'].sum().reset_index()
# Filter out rows where 'year' is 0
song_year_play_count = song_year_play_count[song_year_play_count['year'] != 0]

In [ ]:
# Plot the barplot
plt.figure(figsize=(10, 5))
plt.bar(song_year_play_count['year'], song_year_play_count['play_count'], color='skyblue')
plt.xlabel('Song Year')
plt.xticks(song_year_play_count['year'],rotation=90)  # Ensure all years are shown on x-axis
plt.ylabel('Total Play Count')
plt.title('Total Play Count by Song Year')
plt.show()

Artists having most songs

In [ ]:
# Counting the number of songs for each artist
song_df['artist_name'].value_counts()

In [ ]:
# song artist analysis

# Group by 'artist_name' and count occurrences of 'song_id'
artist_song_counts = song_df.groupby(by='artist_name')['song_id'].count()

# Convert the Series to a DataFrame and reset the index
artist_song_counts = artist_song_counts.reset_index()

# Rename the columns for clarity
artist_song_counts.columns = ['artist_name', 'song_count']

# Set the figure size
plt.figure(figsize=(12, 10))

# Create a histplot with y label as "Number of Artists" and x -axis Song Count

sns.histplot(data=artist_song_counts, x='song_count', binwidth=1)
plt.locator_params(axis='x', nbins=25)

# Set the x label of the plot
plt.xlabel('Song Count')

# Set the y label of the plot
plt.ylabel('Number of Artists')

# Show the plot
plt.title('Song Count Per Artist')
plt.show()

Artists having most users

In [ ]:
user_count_by_artist = df_final.groupby('artist_name')['user_id'].nunique().reset_index()
user_count_by_artist = user_count_by_artist.sort_values(by='user_id', ascending=False)
user_count_by_artist


In [ ]:
plt.figure(figsize=(15, 10))
plt.bar(user_count_by_artist['artist_name'], user_count_by_artist['user_id'], color='skyblue')
plt.xlabel('Artist Name')
plt.ylabel('Count of Unique Users')
plt.title('Count of Unique Users by Artist Name')
plt.xticks(rotation=90, ha='right', fontsize=5)
plt.tight_layout()  # Adjust layout to make room for rotated labels
plt.gca().invert_yaxis()  # Invert y-axis to have the largest bar at the top
plt.show()

Artists having most play count

In [ ]:
play_count_by_artist = df_final.groupby('artist_name')['play_count'].sum().reset_index()
play_count_by_artist = play_count_by_artist.sort_values(by='play_count', ascending=False)
play_count_by_artist


In [ ]:
# Plot the barplot
plt.figure(figsize=(15, 10))
plt.bar(play_count_by_artist['artist_name'], play_count_by_artist['play_count'], color='skyblue')
plt.xlabel('Artist Name')
plt.xticks(play_count_by_artist['artist_name'],rotation=90)  # Ensure all artist name are shown on x-axis
plt.ylabel('Total Play Count')
plt.title('Total Play Count by Artist')
plt.xticks(rotation=90, ha='right', fontsize=5)
plt.tight_layout()  # Adjust layout to make room for rotated labels
plt.gca().invert_yaxis()  # Invert y-axis to have the largest bar at the top
plt.show()

Release having most play count

In [ ]:
release_play_count = df_final.groupby('release')['play_count'].sum().reset_index()
release_play_count = release_play_count.sort_values(by='play_count', ascending=False)
release_play_count

Release having most users

In [ ]:
user_count_by_release = df_final.groupby('release')['user_id'].nunique().reset_index()
user_count_by_release = user_count_by_release.sort_values(by='user_id', ascending=False)
user_count_by_release


Release having most songs (remember in our dataset these songs are already been selected as more popular ones)

In [ ]:
release_song_count = song_df.groupby('release')['song_id'].count().reset_index()
release_song_count = release_song_count.sort_values(by='song_id', ascending=False)
release_song_count

Years having most release

In [ ]:
release_year_count = song_df.groupby('year')['release'].nunique().reset_index()
release_year_count = release_year_count[release_year_count['year'] != 0].sort_values(by='release', ascending=False)
release_year_count

Artists having most release

In [ ]:
artist_release_count = song_df.groupby('artist_name')['release'].nunique().reset_index()
artist_release_count = artist_release_count.sort_values(by='release', ascending=False)
artist_release_count



#### **Observations and Insights:** #



*   The Million Song Dataset released in 2011. That explains most of the songs in the dataset comes from 2000-2010.
*   The artists and their no. of songs follow the power law too. A few popular artists have more songs, while the majority only have relatively low no. of songs.  
*   We do further analysis to find the popular artists and releases.



**Think About It:** What other insights can be drawn using exploratory data analysis?

If have more time, we can consider to do further analysis on play count distribution among artists.

Now that we have explored the data, let's apply different algorithms to build recommendation systems.

**Note:** Use the shorter version of the data, i.e., the data after the cutoffs as used in Milestone 1.

## Building various models

### **Popularity-Based Recommendation Systems**

Let's take the count and sum of play counts of the songs and build the popularity recommendation systems based on the sum of play counts.

In [ ]:
# Calculating average play_count
       # Hint: Use groupby function on the song_id column
average_count = df_final.groupby('song_id')['play_count'].mean()

# Calculating the frequency a song is played
      # Hint: Use groupby function on the song_id column
play_freq = df_final.groupby('song_id')['play_count'].count()

In [ ]:
# Making a dataframe with the average_count and play_freq
final_play = pd.DataFrame({'average_count': average_count, 'play_freq': play_freq})

# Let us see the first five records of the final_play dataset
final_play.head()

Now, let's create a function to find the top n songs for a recommendation based on the average play count of song. We can also add a threshold for a minimum number of playcounts for a song to be considered for recommendation.

In [ ]:
# Build the function to find top n songs based on the highest average rating and minimum interactions
def top_n_songs(data, n, min_interaction = 100):

# Finding products with minimum number of interactions
    recommendations = data[data['play_freq'] > min_interaction]

# Sorting values with respect to average play
    recommendations = data.sort_values(by = 'average_count', ascending = False)

    return recommendations.index[:n]

In [ ]:
# Recommend top 10 songs using the function defined above
list(top_n_songs(final_play, 10))

**Let's define a data frame to save the models' metrics for later comparision.**

In [ ]:
# Define the columns for the DataFrame to save model's info
df_model_info_columns = ['Model Type', 'Parameters', 'Precision', 'Recall', 'F1', 'RMSE', 'Combination']

# Create an empty DataFrame with the specified columns
df_model_info = pd.DataFrame(columns=df_model_info_columns)

# Function to add a new model record to the DataFrame
def add_model_record(df, model_type, parameters, precision, recall, f1, rmse, model_combination=''):
    new_record = {
        'Model Type': model_type,
        'Parameters': parameters,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'RMSE': rmse,
        'Combination': model_combination
    }
    df.loc[len(df)] = new_record

### **User User Similarity-Based Collaborative Filtering**

To build the user-user-similarity-based and subsequent models we will use the "surprise" library.

In [ ]:
# Install the surprise package using pip. Uncomment and run the below code to do the same

!pip install surprise

In [ ]:
# Import necessary libraries

# To compute the accuracy of models
from surprise import accuracy

# This class is used to parse a file containing play_counts, data should be in structure - user; item; play_count
from surprise.reader import Reader

# Class for loading datasets
from surprise.dataset import Dataset

# For tuning model hyperparameters
from surprise.model_selection import GridSearchCV

# For splitting the data in train and test dataset
from surprise.model_selection import train_test_split

# For implementing similarity-based recommendation system
from surprise.prediction_algorithms.knns import KNNBasic

# For implementing matrix factorization based recommendation system
from surprise.prediction_algorithms.matrix_factorization import SVD

# For implementing KFold cross-validation
from surprise.model_selection import KFold

# For implementing clustering-based recommendation system
from surprise import CoClustering

### Some useful functions

Below is the function to calculate precision@k and recall@k, RMSE, and F1_Score@k to evaluate the model performance.

**Think About It:** Which metric should be used for this problem to compare different models?



*   Precision is the proportion of recommended items that are relevant. Higher precision will have better user experience. If users have to sift through many irrelevant recommendations, their satisfaction may decrease.
*   Recall is the proportion of relevant items that are recommended. Higher recall will help users discover as much new and relevant content as possible. It can lead to more engagement, as users find more songs that they enjoy. This is particularly important in competitive markets where retaining user attention is crucial.
*   F1-score is the harmonic mean of precision and recall, providing a single metric that balances the two. It’s particularly useful when both precision and recall are important and a trade-off needs to be considered.
  



In [ ]:
def precision_recall_at_k(model, k=30, threshold=1.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)

    #Making predictions on the test data
    predictions = model.test(testset)

    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, playing_count in user_est_true.items():

        # Sort play count by estimated value
        playing_count.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in playing_count)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in playing_count[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in playing_count[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set Precision to 0 when n_rec_k is 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set Recall to 0 when n_rel is 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    #Mean of all the predicted precisions are calculated.
    precision = round((sum(prec for prec in precisions.values()) / len(precisions)),8)
    #Mean of all the predicted recalls are calculated.
    recall = round((sum(rec for rec in recalls.values()) / len(recalls)),8)
    # Formula to compute the F-1 score.
    f1 = round((2*precision*recall)/(precision+recall),8)
    # Retrieve RMSE and print
    rmse = round(accuracy.rmse(predictions),8)

    print('Precision: ', precision) #Command to print the overall precision
    print('Recall: ', recall) #Command to print the overall recall
    print('F_1 score: ', f1) #Command to print the F-1 score

    # Return all 4 scores
    return precision, recall, f1, rmse

**Think About It:** In the function precision_recall_at_k above the threshold value used is 1.5. How precision and recall are affected by changing the threshold? What is the intuition behind using the threshold value of 1.5?



*   Precision: Precision measures the proportion of recommended items that are relevant to the user out of all recommended items. When the threshold is increased, it becomes harder for items to be considered relevant, leading to potentially fewer true positives (relevant items) being recommended. However, because the number of false positives (irrelevant items) decreases more significantly, the precision typically increases as the threshold increases, making the algorithm more conservative in its recommendations.

*   Recall: Recall measures the proportion of relevant items that are successfully recommended out of all relevant items in the dataset. Increasing the threshold causes the algorithm to be more selective in recommending items, potentially missing some relevant items that would have been recommended with a lower threshold. This leads to an increase in false negatives and consequently a decrease in recall as the threshold increases, because the algorithm may overlook some relevant items.

*   The selection of the threshold value, specifically 1.5 in our case, corresponds to the play count criterion within our dataset. In above data analysis, the majority fall between 1 and 2, a threshold of 1.5 helps filter out items that are not strongly relevant while keeping those that are most likely to be of interest. Given that this threshold represents a play count above which a song is considered significant (rounded to 2 for practical application), it signifies instances where a user has played a song more than once. The rationale behind this choice stems from the recognition of our dataset's sparsity, necessitating a cautious balance in recommendation strategies. Opting for a threshold lower than 1.5 could potentially result in an overabundance of irrelevant song recommendations, which would compromise the effectiveness of our recommendation system.



Below we are loading the **dataset**, which is a **pandas dataframe**, into a **different format called `surprise.dataset.DatasetAutoFolds`** which is required by this library. To do this we will be **using the classes `Reader` and `Dataset`**

You will also notice here that we read the dataset by providing a scale of ratings. However, as you would know, we do not have ratings data of the songs. In this case, we are going to use play_count as a proxy for ratings with the assumption that the more the user listens to a song, the higher the chance that they like the song.

In [ ]:
# Instantiating Reader scale with expected rating scale
 #use rating scale (0, 5)
reader = Reader(rating_scale = (0, 5))

# Loading the dataset
 # Take only "user_id","song_id", and "play_count"
data = Dataset.load_from_df(df_final[['user_id', 'song_id', 'play_count']], reader)

# Splitting the data into train and test dataset
 # Take test_size = 0.4, random_state = 42
trainset, testset = train_test_split(data, test_size = 0.4, random_state = 42)

**Think About It:** How changing the test size would change the results and outputs?



*   As the test size increases, the proportion of data available for training decreases. This could potentially impact the model's ability to learn from the training data, especially if the training set becomes too small. With a smaller training set, the model may not capture the underlying patterns in the data as effectively, which could lead to poorer performance.

*   But a larger test size means more data is held out for evaluation, providing a more reliable estimate of how well the model will perform on unseen data.

*   A smaller test size may lead to higher variance because the evaluation of model performance is more sensitive to the specific random split of the data. Conversely, a larger test size may lead to higher bias because the model has less data for training and may generalize less effectively to unseen data.

In summary, changing the test size when splitting the data can impact the trade-off between training and testing performance, generalization ability, bias and variance, and the robustness of the results.


In [ ]:
# Build the default user-user-similarity model
sim_options = {'name': 'cosine',
               'user_based': True}

# KNN algorithm is used to find desired similar items
 # Use random_state = 1
sim_user_user = KNNBasic(sim_options = sim_options, verbose = False, random_state = 1)

# Train the algorithm on the trainset, and predict play_count for the testset
sim_user_user.fit(trainset)

In [ ]:
# Let us compute precision@k, recall@k, and f_1 score with k = 30
# Use sim_user_user model
precision, recall, f1, rmse = precision_recall_at_k(sim_user_user)
# Save the model's info
sim_user_user_params = f"k={sim_user_user.k}, min_k={sim_user_user.min_k}, sim_options={sim_user_user.sim_options}, verbose={sim_user_user.verbose}"
add_model_record(df_model_info, 'User-User', sim_user_user_params, precision, recall, f1, rmse)

In [ ]:
# Check the model info dataframe
df_model_info

**Observations and Insights:**



- We have calculated **RMSE** to check **how far the overall predicted play counts** are from the **actual play counts**.
- Intuition of Recall: We are getting a **recall of ~0.692**, which means out of **all the relevant songs 69.2% are recommended**.
- Intuition of Precision: We are getting a **precision of ~ 0.396**, which means **out of all the recommended songs 39.6% are relevant**.
- Here **F_1 score** of the **baseline model is ~0.504**. It indicates that **the model is performing moderately**, precision and recall are balanced. The model is reasonably good at minimizing false positives (precision) and identifying relevant instances (recall) but does not excel in either aspect.
- We will try to improve this later by using **GridSearchCV by tuning different hyperparameters** of this algorithm.
- We create a dataframe to save the model's info for later comparision.



In [ ]:
# Predicting play_count for a sample user with a listened song
# Use any user id  and song_id
sim_user_user.predict(6958, 1671, r_ui = 2, verbose = True)

In [ ]:
# Predicting play_count for a sample user with a song not-listened by the user
 #predict play_count for any sample user
sim_user_user.predict(6958, 3232, verbose = True)

**Observations and Insights:**


- The above output shows that **the actual play count 2 is close to the predicted play count 1.8 for this user-item pair** by this **user-user-similarity-based baseline model**.
- The **output** also contains **"actual_k"**. It is the value of **K in KNN** that is used while training the model. The default value is 40.
- Above, we also **predicted the play count for a non-interacted user-item pair** based on this user-user similarity-based baseline model.



Now, let's try to tune the model and see if we can improve the model performance.

In [ ]:
# Setting up parameter grid to tune the hyperparameters
param_grid = {'k': [30, 40, 50], 'min_k': [3, 6, 9],
              'sim_options': {'name': ['msd', 'cosine'],
                              'user_based': [True]}
              }

# Performing 3-fold cross-validation to tune the hyperparameters
gs = GridSearchCV(KNNBasic, param_grid, measures = ['rmse'], cv = 3, n_jobs = -1)

# Fitting the data
 # Use entire data for GridSearch
gs.fit(data)

# Best RMSE score
print(gs.best_score['rmse'])

# Combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

In [ ]:
# Train the best model found in above gridsearch

# Using the optimal similarity measure for user-user based collaborative filtering
sim_options = {'name': 'msd',
               'user_based': True}

# Creating an instance of KNNBasic with optimal hyperparameter values
sim_user_user_optimized = KNNBasic(sim_options = sim_options, k = 50, min_k = 9, random_state = 1, verbose = False)

# Training the algorithm on the trainset
sim_user_user_optimized.fit(trainset)

In [ ]:
# Let us compute precision@k and recall@k also with k =30
precision, recall, f1, rmse = precision_recall_at_k(sim_user_user_optimized)

# Save the model's info
sim_user_user_optimized_params = f"k={sim_user_user_optimized.k}, min_k={sim_user_user_optimized.min_k}, sim_options={sim_user_user_optimized.sim_options}, verbose={sim_user_user_optimized.verbose}"
add_model_record(df_model_info, 'User-User Optimized', sim_user_user_optimized_params, precision, recall, f1, rmse)

**Observations and Insights:**

- We can see from above that after tuning hyperparameters, **the RMSE of the model has gone down as compared to the model before hyperparameter tuning**. Along with this, **Precision score of the tuned model has increased in comparison to the baseline model**, but recall and F1 score went down. The model performance didn't improve much after hyperparameter tuning.

In [ ]:
# Predict the play count for a user who has listened to the song. Take user_id 6958, song_id 1671 and r_ui = 2
sim_user_user_optimized.predict(6958, 1671, r_ui = 2, verbose = True)

In [ ]:
# Predict the play count for a song that is not listened to by the user (with user_id 6958)
sim_user_user_optimized.predict(6958, 3232, verbose = True)

**Observations and Insights:**

- The predicted play count by the tuned model varies from the prediction by the baseline model, deviating notably from the actual play count.

**Think About It:** Along with making predictions on listened and unknown songs can we get 5 nearest neighbors (most similar) to a certain song?

In [ ]:
# Use inner id 0
sim_user_user_optimized.get_neighbors(0, k = 5)

Below we will be implementing a function where the input parameters are:

- data: A **song** dataset
- user_id: A user-id **against which we want the recommendations**
- top_n: The **number of songs we want to recommend**
- algo: The algorithm we want to use **for predicting the play_count**
- The output of the function is a **set of top_n items** recommended for the given user_id based on the given algorithm

In [ ]:
def get_recommendations(data, user_id, top_n, algo):

    # Creating an empty list to store the recommended song ids
    recommendations = []

    # Creating an user item interactions matrix
    user_item_interactions_matrix = data.pivot(index = 'user_id', columns = 'song_id', values = 'play_count')

    # Extracting those song ids which the user_id has not played yet
    non_interacted_songs = user_item_interactions_matrix.loc[user_id][user_item_interactions_matrix.loc[user_id].isnull()].index.tolist()

    # Looping through each of the song ids which user_id has not interacted yet
    for song_id in non_interacted_songs:

        # Predicting the users for those non played song ids by this user
        est = algo.predict(user_id, song_id).est

        # Appending the predicted play_counts
        recommendations.append((song_id, est))

    # Sorting the predicted play_counts in descending order
    recommendations.sort(key = lambda x: x[1], reverse = True)

    return recommendations[:top_n] # Returing top n highest predicted play_count songs for this user

In [ ]:
# Make top 5 recommendations for any user_id with a similarity-based recommendation engine
recommendations = get_recommendations(df_final, 6958, 5, sim_user_user_optimized)

In [ ]:
# Building the dataframe for above recommendations with columns "song_id" and "predicted_play_count"
pd.DataFrame(recommendations, columns = ['song_id', 'predicted_play_count'])

**Observations and Insights:**



*   We implement a function to Make top n recommendations for any user id with a fitted model. It will return the song ids and predicted play counts.




### Correcting the play_counts and Ranking the above songs

In [ ]:
def ranking_songs(recommendations, final_play):
  # Sort the songs based on play counts
  ranked_songs = final_play.loc[[items[0] for items in recommendations]].sort_values('play_freq', ascending = False)[['play_freq']].reset_index()

  # Merge with the recommended songs to get predicted play_counts
  ranked_songs = ranked_songs.merge(pd.DataFrame(recommendations, columns = ['song_id', 'predicted_play_count']), on = 'song_id', how = 'inner')

  # Rank the songs based on corrected play_counts
  ranked_songs['corrected_play_count'] = ranked_songs['predicted_play_count'] - 1 / np.sqrt(ranked_songs['play_freq'])

  # Sort the songs based on corrected play_counts
  ranked_songs = ranked_songs.sort_values('corrected_play_count', ascending = False)

  return ranked_songs

**Think About It:** In the above function to correct the predicted play_count a quantity 1/np.sqrt(n) is subtracted. What is the intuition behind it? Is it also possible to add this quantity instead of subtracting?



*   By subtracting 1/np.sqrt(n), the model is penalized slightly, which helps to prevent overfitting. When the number of observations is small, there's a higher risk of overfitting because the model may try to fit the noise in the data rather than capturing the underlying patterns. It can be seen as a form of bias correction. It helps to adjust the predictions to be more conservative, especially when dealing with limited data. And it makes the predictions more robust by reducing the influence of outliers or extreme values.

*   It's possible to add instead of subtracting. Adding would have a similar effect of regularizing the predictions and biasing them slightly, but in the opposite direction. It will get more optimistic predictions.



In [ ]:
# Applying the ranking_songs function on the final_play data
ranking_songs(recommendations, final_play)

**Observations and Insights:**



*   After the correction, the predicted play counts all decrease slightly. The magnitude of this decrease is proportional to the play frequency.



### Item Item Similarity-based collaborative filtering recommendation systems

In [ ]:
# Apply the item-item similarity collaborative filtering model with random_state = 1 and evaluate the model performance
sim_options = {'name': 'cosine',
               'user_based': False}

# The KNN algorithm is used to find desired similar items
sim_item_item = KNNBasic(sim_options = sim_options, random_state = 1, verbose = False)

# Train the algorithm on the train set, and predict ratings for the testset
sim_item_item.fit(trainset)

In [ ]:
# Let us compute precision@k and recall@k also with k =30
precision, recall, f1, rmse = precision_recall_at_k(sim_item_item)

# Save the model's info
sim_item_item_params = f"k={sim_item_item.k}, min_k={sim_item_item.min_k}, sim_options={sim_item_item.sim_options}, verbose={sim_item_item.verbose}"
add_model_record(df_model_info, 'Item-Item', sim_item_item_params, precision, recall, f1, rmse)

**Observations and Insights:**

- Here, **F_1 score** of the **baseline model** is **~0.397**. We will try to improve this later by tuning different hyperparameters of this algorithm using **GridSearchCV**.

In [ ]:
# Predicting play count for a sample user_id 6958 and song (with song_id 1671) listened to by the user
sim_item_item.predict(6958, 1671, r_ui = 2, verbose = True)

In [ ]:
# Predict the play count for a user that has not listened to the song (with song_id 1671)
sim_item_item.predict(67704, 1671, verbose = True)

In [ ]:
# Predict the play count for a song that is not listened to by the user (with user_id 6958)
sim_item_item.predict(6958, 3232, verbose = True)

**Observations and Insights:**

- The predicted play count by this model deviate notably from the actual play count.

In [ ]:
# Apply grid search for enhancing model performance

# Setting up parameter grid to tune the hyperparameters
param_grid = {'k': [30, 40, 50], 'min_k': [3, 6, 9],
              'sim_options': {'name': ['msd', 'cosine'],
                              'user_based': [False]}
              }

# Performing 3-fold cross-validation to tune the hyperparameters
gs = GridSearchCV(KNNBasic, param_grid, measures = ['rmse'], cv = 3, n_jobs = -1)

# Fitting the data
gs.fit(data)

# Find the best RMSE score
print(gs.best_score['rmse'])

# Extract the combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

**Think About It:** How do the parameters affect the performance of the model? Can we improve the performance of the model further? Check the list of hyperparameters [here](https://surprise.readthedocs.io/en/stable/knn_inspired.html).

In [ ]:
# Apply the best model found in the grid search
sim_options = {'name': 'msd',
               'user_based': False}

# Creating an instance of KNNBasic with optimal hyperparameter values
sim_item_item_optimized = KNNBasic(sim_options = sim_options, k = 40, min_k = 6, random_state = 1, verbose = False)

# Training the algorithm on the train set
sim_item_item_optimized.fit(trainset)

In [ ]:
# Let us compute precision@k and recall@k also with k =30
precision, recall, f1, rmse = precision_recall_at_k(sim_item_item_optimized)

# Save the model's info
sim_item_item_optimized_params = f"k={sim_item_item_optimized.k}, min_k={sim_item_item_optimized.min_k}, sim_options={sim_item_item_optimized.sim_options}, verbose={sim_item_item_optimized.verbose}"
add_model_record(df_model_info, 'Item-Item Optimized', sim_item_item_optimized_params, precision, recall, f1, rmse)

**Observations and Insights:**

- We can observe that after tuning hyperparameters, **F_1 score, Precision and Recall of the model are all slightly better than the baseline model**. Along with this, **the RMSE of the model has gone down a little bit in comparison to the model with default hyperparameters**. Hence, we can say that the model performance has improved after hyperparameter tuning.

In [ ]:
# Predict the play_count by a user(user_id 6958) for the song (song_id 1671)
sim_item_item_optimized.predict(6958, 1671, r_ui = 2, verbose = True)

In [ ]:
# Predicting play count for a sample user_id 6958 with song_id 3232 which is not listened to by the user
sim_item_item_optimized.predict(6958, 3232, verbose = True)

**Observations and Insights:**

- The predicted play count are exactly the same as the baseline model. Although the overall score has increased a little bit, the prediction for this particular user is still not that good.

In [ ]:
# Find five most similar items to the item with inner id 0
sim_item_item_optimized.get_neighbors(0, k = 5)

In [ ]:
# Making top 5 recommendations for any user_id  with item_item_similarity-based recommendation engine
recommendations = get_recommendations(df_final, 6958, 5, sim_item_item_optimized)

In [ ]:
# Building the dataframe for above recommendations with columns "song_id" and "predicted_play_count"
pd.DataFrame(recommendations, columns = ['song_id', 'predicted_play_count'])

In [ ]:
# Applying the ranking_songs function
ranking_songs(recommendations, final_play)

**Observations and Insights:**

*   We use the make top n recommendations function for the same user with the new fitted model. It recommended different songs.
*   The ranking_song function is used on this new set of recommended songs and with the corrected play count their ranking order are different.

### Model Based Collaborative Filtering - Matrix Factorization

Model-based Collaborative Filtering is a **personalized recommendation system**, the recommendations are based on the past behavior of the user and it is not dependent on any additional information. We use **latent features** to find recommendations for each user.

In [ ]:
# Build baseline model using svd

# Using SVD with matrix factorization
svd = SVD(random_state = 1)

# Training the algorithm on the training dataset
svd.fit(trainset)

In [ ]:
# Let us compute precision@k and recall@k also with k =30
precision, recall, f1, rmse = precision_recall_at_k(svd)

# Save the model's info
svd_params = f"'n_epochs':{svd.n_epochs}, 'lr':{svd.lr_bu}, 'reg':{svd.reg_bu}"
add_model_record(df_model_info, 'SVD', svd_params, precision, recall, f1, rmse)

In [ ]:
# Making prediction for user (with user_id 6958) to song (with song_id 1671), take r_ui = 2
svd.predict(6958, 1671, r_ui = 2, verbose = True)

In [ ]:
# Making a prediction for the user who has not listened to the song (song_id 3232)
svd.predict(6958, 3232, verbose = True)

#### Improving matrix factorization based recommendation system by tuning its hyperparameters

In [ ]:
# Set the parameter space to tune
param_grid = {'n_epochs': [10, 20, 30], 'lr_all': [0.001, 0.005, 0.01],
              'reg_all': [0.2, 0.4, 0.6]}

# Performe 3-fold grid-search cross-validation
gs = GridSearchCV(SVD, param_grid, measures = ['rmse'], cv = 3, n_jobs = -1)

# Fitting data
gs.fit(data)

# Best RMSE score
print(gs.best_score['rmse'])

# Combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

**Think About It**: How do the parameters affect the performance of the model? Can we improve the performance of the model further? Check the available hyperparameters [here](https://surprise.readthedocs.io/en/stable/matrix_factorization.html).


*   The number of iterations (epochs) controls the number of times the model updates its parameters based on the training data. It can affect the model's convergence, risk of overfitting, computational cost, and overall performance.

*   The learning rate determines the size of the steps taken during optimization. A too high or too low learning rate can lead to convergence issues or slow convergence, respectively. Optimizing the learning rate is crucial for achieving good performance.

*   Regularization strength for all parameters helps prevent overfitting by penalizing large parameter values. A higher regularization strength encourages simpler models with smaller parameter values, reducing the risk of overfitting. The choice of an appropriate regularization strength depends on the specific dataset and the trade-off between bias and variance.



In [ ]:
# Building the optimized SVD model using optimal hyperparameters
svd_optimized = SVD(n_epochs = 30, lr_all = 0.01, reg_all = 0.2, random_state = 1)

# Training the algorithm on the train set
svd_optimized = svd_optimized.fit(trainset)

In [ ]:
# Let us compute precision@k and recall@k also with k =30
precision, recall, f1, rmse = precision_recall_at_k(svd_optimized)

# Save the model's info
svd_optimized_params = gs.best_params
add_model_record(df_model_info, 'SVD Optimized', svd_optimized_params, precision, recall, f1, rmse)

**Observations and Insights:**

- We can observe that after tuning hyperparameters, **F_1 score, Precision and Recall of the model are all slightly better than the baseline model**. Along with this, **the RMSE of the model has gone down a little bit in comparison to the model with default hyperparameters**. Hence, we can say that the model performance has improved after hyperparameter tuning.

In [ ]:
# Using svd_algo_optimized model to recommend for userId 6958 and song_id 1671
svd_optimized.predict(6958, 1671, r_ui = 2, verbose = True)

In [ ]:
# Using svd_algo_optimized model to recommend for userId 6958 and song_id 3232 with unknown baseline play_count
svd_optimized.predict(6958, 3232, verbose = True)

**Observations and Insights:**



*   The predicted play counts are close to the baseline model. Although the overall score has increased a little bit, the prediction for this particular user is still not that good.




In [ ]:
# Getting top 5 recommendations for user_id 6958 using "svd_optimized" algorithm
svd_recommendations = get_recommendations(df_final, 6958, 5, svd_optimized)

In [ ]:
# Ranking songs based on above recommendations
ranking_songs(svd_recommendations, final_play)

**Observations and Insights:*

*   We use the make top n recommendations function for the same user with the new fitted model. It recommended different songs.
*   The ranking_song function is used on this new set of recommended songs and returned the corrected play count.

### Cluster Based Recommendation System

In **clustering-based recommendation systems**, we explore the **similarities and differences** in people's tastes in songs based on how they rate different songs. We cluster similar users together and recommend songs to a user based on play_counts from other users in the same cluster.

In [ ]:
# Make baseline clustering model

# Using CoClustering algorithm
clust_baseline = CoClustering(random_state = 1)

# Training the algorithm on the train set
clust_baseline.fit(trainset)

In [ ]:
# Let us compute precision@k and recall@k also with k =30
precision, recall, f1, rmse = precision_recall_at_k(clust_baseline)

# Save the model's info
clust_baseline_params = f"n_cltr_u: {clust_baseline.n_cltr_u}, n_cltr_i: {clust_baseline.n_cltr_i}, n_epochs: {clust_baseline.n_epochs}"
add_model_record(df_model_info, 'Cluster Based', clust_baseline_params, precision, recall, f1, rmse)

In [ ]:
# Making prediction for user_id 6958 and song_id 1671
clust_baseline.predict(6958, 1671, r_ui = 2, verbose = True)

In [ ]:
# Making prediction for user (userid 6958) for a song(song_id 3232) not listened to by the user
clust_baseline.predict(6958, 3232, verbose = True)

#### Improving clustering-based recommendation system by tuning its hyper-parameters

In [ ]:
# Set the parameter space to tune
param_grid = {'n_cltr_u': [3, 4, 5, 6], 'n_cltr_i': [3, 4, 5, 6], 'n_epochs': [30, 40, 50]}

# Performing 3-Fold gridsearch cross-validation
gs = GridSearchCV(CoClustering, param_grid, measures = ['rmse'], cv = 3, n_jobs = -1)

# Fitting data
gs.fit(data)

# Printing the best RMSE score
print(gs.best_score['rmse'])

# Printing the combination of parameters that gives the best RMSE score
print(gs.best_params['rmse'])

**Think About It**: How do the parameters affect the performance of the model? Can we improve the performance of the model further? Check the available hyperparameters [here](https://surprise.readthedocs.io/en/stable/co_clustering.html).

In [ ]:
# Train the tuned Coclustering algorithm
# Using tuned Coclustering algorithm
clust_tuned = CoClustering(n_cltr_u = 3, n_cltr_i = 5, n_epochs = 50, random_state = 1)

# Training the algorithm on the train set
clust_tuned.fit(trainset)

In [ ]:
# Let us compute precision@k and recall@k also with k =30
precision, recall, f1, rmse = precision_recall_at_k(clust_tuned)

# Save the model's info
clust_tuned_params = gs.best_params
add_model_record(df_model_info, 'Cluster Tuned', clust_tuned_params, precision, recall, f1, rmse)

In [ ]:
# Check the model info dataframe
df_model_info

**Observations and Insights:**


- We can observe that after tuning hyperparameters, **the RMSE, F_1 score, Precision and Recall of the model are all the same as the baseline model**.




In [ ]:
# Using co_clustering_optimized model to recommend for userId 6958 and song_id 1671
clust_tuned.predict(6958, 1671, r_ui = 2, verbose = True)

In [ ]:
# Use Co_clustering based optimized model to recommend for userId 6958 and song_id 3232 with unknown baseline play_count
clust_tuned.predict(6958, 3232, verbose = True)

**Observations and Insights:**

-   The prediction for this particular user is still not that good.

#### Implementing the recommendation algorithm based on optimized CoClustering model

In [ ]:
# Getting top 5 recommendations for user_id 6958 using "Co-clustering based optimized" algorithm
clustering_recommendations = get_recommendations(df_final, 6958, 5, clust_tuned)

### Correcting the play_count and Ranking the above songs

In [ ]:
# Ranking songs based on the above recommendations
ranking_songs(clustering_recommendations, final_play)

**Observations and Insights:**

*   We use the make top n recommendations function for the same user with the new fitted model. It recommended different songs. But some songs are overlapping.
*   The ranking_song function is used on this new set of recommended songs and returned the corrected play count. The prediction to the same song are similar to the previous model's.

### Content Based Recommendation Systems

**Think About It:** So far we have only used the play_count of songs to find recommendations but we have other information/features on songs as well. Can we take those song features into account?

In [ ]:
# Concatenate the "title", "release", "artist_name" columns to create a different column named "text"
df_final['text'] = df_final['title'] + ' ' + df_final['release'] + ' ' + df_final['artist_name']

df_final.head()

In [ ]:
# Select the columns 'user_id', 'song_id', 'play_count', 'title', 'text' from df_small data
df_small = df_final[['user_id', 'song_id', 'play_count', 'title', 'text']]

# Drop the duplicates from the title column
df_small = df_small.drop_duplicates(subset = ['title'])

# Set the title column as the index
df_small = df_small.set_index('title')

# See the first 5 records of the df_small dataset
df_small.head()

In [ ]:
# Create the series of indices from the data
indices = pd.Series(df_small.index)

In [ ]:
# Importing necessary packages to work with text data
import nltk
nltk.download('omw-1.4')

# Download punkt library
nltk.download('punkt')

# Download stopwords library
nltk.download('stopwords')

# Download wordnet
nltk.download('wordnet')

# Import regular expression
import re

# Import word_tokenizer
from nltk import word_tokenize

# Import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer

# Import stopwords
from nltk.corpus import stopwords

# Import CountVectorizer and TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

We will create a **function to pre-process the text data:**

In [ ]:
# Create a function to tokenize the text
def tokenize(text):

    # Making each letter as lowercase and removing non-alphabetical text
    text = re.sub(r"[^a-zA-Z]"," ", text.lower())

    # Extracting each word in the text
    tokens = word_tokenize(text)

    # Removing stopwords
    words = [word for word in tokens if word not in stopwords.words("english")]

    # Lemmatize the words
    text_lems = [WordNetLemmatizer().lemmatize(lem).strip() for lem in words]

    return text_lems

In [ ]:
# Create tfidf vectorizer
tfidf = TfidfVectorizer(tokenizer = tokenize)

# Fit_transfrom the above vectorizer on the text column and then convert the output into an array
song_tfidf = tfidf.fit_transform(df_small['text'].values).toarray()

In [ ]:
# Compute the cosine similarity for the tfidf above output
similar_songs = cosine_similarity(song_tfidf, song_tfidf)

# Let us see the above array
similar_songs

 Finally, let's create a function to find most similar songs to recommend for a given song.

In [ ]:
# Function that takes in song title as input and returns the top 10 recommended songs
def recommendations(title, similar_songs):

    recommended_songs = []

    indices = pd.Series(df_small.index)

    # Getting the index of the song that matches the title
    idx = indices[indices == title].index[0]

    # Creating a Series with the similarity scores in descending order
    score_series = pd.Series(similar_songs[idx]).sort_values(ascending = False)

    # Getting the indexes of the 10 most similar songs
    top_10_indexes = list(score_series.iloc[1 : 11].index)
    print(top_10_indexes)

    # Populating the list with the titles of the best 10 matching songs
    for i in top_10_indexes:
        recommended_songs.append(list(df_small.index)[i])

    return recommended_songs


Recommending 10 songs similar to Learn to Fly

In [ ]:
# Make the recommendation for the song with title 'Learn To Fly'
recommendations('Learn To Fly', similar_songs)

**Observations and Insights:**

- The song 'Learn To Fly' belongs to **Rock** genres, and the **majority of our recommendations** lie in this genres. It implies that the resulting recommendation system is working well.

### Hybrid Recommendation Systems

When dealing with sparse datasets in recommendation systems, combining the predictions from multiple models to create a hybrid model can improve performance. Below we will be implementing 3 different regression methods to build hybrid recommendation systems.  

In [ ]:
# Importing libraries for building linear regression model
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV as SklearnGridSearchCV

In [ ]:
# Extract predictions from models
def hybrid_extract(models, dataset):

    # Dictionary to store results
    pred_dict = {}

    for model_index, model in enumerate(models):

        # Get predictions for the current model
        predictions = model.test(dataset)

        # Loop through each prediction
        for pred in predictions:
            uid = pred.uid
            iid = pred.iid
            est = pred.est
            act = pred.r_ui

            # Create a unique key for each uid-iid pair
            key = (uid, iid)

            # Initialize the dictionary entry if it doesn't exist
            if key not in pred_dict:
                pred_dict[key] = {'user_id': uid, 'song_id': iid}
                pred_dict[key]['actual'] = act

            # Store the model's prediction in the dictionary
            pred_dict[key][f'prediction_{model_index+1}'] = est


    # Convert to DataFrame
    df_pred = pd.DataFrame.from_dict(pred_dict, orient='index')

    # Reset index to flatten the DataFrame
    df_pred.reset_index(drop=True, inplace=True)


    # Prepare for later regression

    # Extract only the model prediction columns
    prediction_columns = [col for col in df_pred.columns if col.startswith('prediction_')]
    X = df_pred[prediction_columns]

    y = df_pred['actual']

    return X,y,df_pred

In [ ]:
# Mark down Regression results and Evaluate the performance of the regression models
def hybrid_process(X, dataset, regression_model):

    # Predict play count using the hybrid model
    dataset['hybrid_prediction'] = regression_model.predict(X)

    # Calc mse of the regression models
    mse = round(mean_squared_error(dataset['actual'], dataset['hybrid_prediction']),3)

    # Print mse of the regression models
    print('---------------------------------------------------------------------------------------------------------')
    print(f'Regression Mean Squared Error: {mse}')

    # Print coefficients and intercept
    print(f'Regression Coefficients: {regression_model.coef_}')
    print(f'Regression Intercept   : {regression_model.intercept_}')

    return dataset


In [ ]:
# Calc precision@k, recall@k, and F_1 score
def hybrid_precision_recall_at_k(df_pred, k=30, threshold=1.5):
    """Return precision and recall and F_1 score at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)

    for index, row in df_pred.iterrows():
        uid = row['user_id']
        true_r = row['actual']
        est = row['hybrid_prediction']
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, playing_count in user_est_true.items():

        # Sort play count by estimated value
        playing_count.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in playing_count)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in playing_count[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in playing_count[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set Precision to 0 when n_rec_k is 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set Recall to 0 when n_rel is 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    #Mean of all the predicted precisions are calculated.
    precision = round((sum(prec for prec in precisions.values()) / len(precisions)),8)
    #Mean of all the predicted recalls are calculated.
    recall = round((sum(rec for rec in recalls.values()) / len(recalls)),8)
    # Formula to compute the F-1 score.
    f1 = round((2*precision*recall)/(precision+recall),8)


    # Calculate RMSE for the predictions
    y_true = df_pred['actual']
    y_pred = df_pred['hybrid_prediction']

    rmse = round(mean_squared_error(y_true, y_pred, squared=False),8)

    print('---------------------------------------------------------------------------------------------------------')
    print('RMSE: ', rmse)  # Print the RMSE
    print('Precision: ', precision) #Command to print the overall precision
    print('Recall: ', recall) #Command to print the overall recall
    print('F_1 score: ', f1) # Formula to compute the F-1 score.
    print('---------------------------------------------------------------------------------------------------------')

    # Return all 4 scores
    return precision, recall, f1, rmse

In [ ]:
# Check predictions for a user with an song
def hybrid_predict(models, uid, iid, regression_model, verbose=True, pca_model=None):
    # verbose to control to print the preditions or not
    # Initialize the new record with uid and iid
    new_record = {
    'uid': uid,
    'iid': iid,
    'hybrid_prediction': 0
    }

    # Loop through the models and add the predictions to the new record
    for model_index, model in enumerate(models):
        prediction_key = f'prediction_{model_index + 1}'
        new_record[prediction_key] = model.predict(uid, iid).est


    # Convert to data frame
    df_merged1 = pd.DataFrame([new_record])
    # Prepare data for linear regression
    prediction_columns = [col for col in df_merged1.columns if col.startswith('prediction_')]
    X1 = df_merged1[prediction_columns]

    if pca_model is not None:
      # transform using the fitted PCA
      X1_pca = pca.transform(X1)
      # Predict ratings using the hybrid model
      regression_model_pred = regression_model.predict(X1_pca)
    else:
      # Predict ratings using the hybrid model
      regression_model_pred = regression_model.predict(X1)

    df_merged1['hybrid_prediction'] = regression_model_pred

    # Display the DataFrame with hybrid predictions
    if (verbose==True):
        print(df_merged1)

    return regression_model_pred


In [ ]:
def hybrid_get_recommendations(data, user_id, top_n, models, regression_model, pca_model=None):

    # Creating an empty list to store the recommended song ids
    recommendations = []

    # Creating an user item interactions matrix
    user_item_interactions_matrix = data.pivot(index = 'user_id', columns = 'song_id', values = 'play_count')

    # Extracting those song ids which the user_id has not played yet
    non_interacted_songs = user_item_interactions_matrix.loc[user_id][user_item_interactions_matrix.loc[user_id].isnull()].index.tolist()

    # Looping through each of the song ids which user_id has not interacted yet
    for song_id in non_interacted_songs:

        # Predicting the users for those non played song ids by this user
        est = hybrid_predict(models, user_id, song_id, regression_model, verbose=False, pca_model=pca_model)

        # Appending the predicted play_counts
        recommendations.append((song_id, est))

    # Sorting the predicted play_counts in descending order
    recommendations.sort(key = lambda x: x[1], reverse = True)

    return recommendations[:top_n] # Returing top n highest predicted play_count songs for this user

Let's try to combine the 4 optimized models.

In [ ]:
# Combine all models
models = [svd_optimized, sim_user_user_optimized, sim_item_item_optimized, clust_tuned]
model_combination = 'SVD, U-U, I-I, Co-Clustering'

# Get predictions from models for the training set
trainset_tuples = [(trainset.to_raw_uid(iuid), trainset.to_raw_iid(iiid), trainset.ur[iuid][jj][1])
                for (iuid, iiratings) in trainset.ur.items()
                for jj, (iiid, _) in enumerate(iiratings)]

X_train, y_train, df_trainset_pred = hybrid_extract(models, trainset_tuples)
# Create a deep copy to backup the init. value
df_trainset_pred_init = df_trainset_pred.copy(deep=True)

# Get predictions from models for the test set
X_test, y_test, df_testset_pred = hybrid_extract(models, testset)
# Create a deep copy to backup the init. value
df_testset_pred_init = df_testset_pred.copy(deep=True)

In [ ]:
# Show the correlation of the predictions
plt.figure(figsize=(10,5))
sns.heatmap(X_train.corr(),annot=True,cmap='Spectral',vmin=-1,vmax=1)
plt.show()

Use PCA to remove correlation before applying linear regression.

In [ ]:
# Handle correlation with PCA
pca = PCA(n_components=len(models))
X_train_pca = pca.fit_transform(X_train)
print("Explained variance ratio:", pca.explained_variance_ratio_)
print("Total explained variance:", np.sum(pca.explained_variance_ratio_))
X_test_pca = pca.transform(X_test)

In [ ]:
# Convert PCA result to DataFrame
X_train_pca_df = pd.DataFrame(X_train_pca, columns=[f'PC{i+1}' for i in range(X_train_pca.shape[1])])
# Show the correlation after PCA
plt.figure(figsize=(10,5))
sns.heatmap(X_train_pca_df.corr(),annot=True,cmap='Spectral',vmin=-1,vmax=1)
plt.show()

After applying PCA, the resulting principal components are orthogonal to each other, effectively removing any correlations present in the original feature space.

#### Hybrid - Linear Regression recommendation system

In [ ]:
# Get predictions from models for the training set (recover from backup)
df_trainset_pred = df_trainset_pred_init

# Get predictions from models for the test set (recover from backup)
df_testset_pred = df_testset_pred_init

# Train a linear regression model using the PCA components
lr = LinearRegression()
lr.fit(X_train_pca, y_train)

df_trainset_pred = hybrid_process(X_train_pca, df_trainset_pred, lr)

# Predict play count using the hybrid model
print('---------------------------------------------------------------------------------------------------------')
print('Apply on Test Set:')
df_testset_pred = hybrid_process(X_test_pca, df_testset_pred, lr)

In [ ]:
# Calc precision@k, recall@k, and F_1 score
precision, recall, f1, rmse = hybrid_precision_recall_at_k(df_testset_pred, k=30, threshold=1.5)

# Save the model's info
Hybrid_params = lr.get_params()
add_model_record(df_model_info, 'Hybrid - PCA and Linear Regression', Hybrid_params, precision, recall, f1, rmse, model_combination)

The Recall score has improved greatly! It increased to 0.819! Let's look into the detailed predictions for both train set and test set.

In [ ]:
df_trainset_pred

In [ ]:
df_testset_pred

In [ ]:
# Check predictions for a user with an interacted song
h_pred = hybrid_predict(models, 6958, 1671, lr, pca_model=pca)

It looks pretty good. The PCA and Linear Regression predicted 1.8~ is very close the actual 2.0 play count.

In [ ]:
# Check predictions for a user with an not-interacted song
h_pred = hybrid_predict(models, 6958, 3232, lr, pca_model=pca)

In [ ]:
# Making top 5 recommendations for a user_id with hybrid recommendation engine
hybrid_recommendations = hybrid_get_recommendations(df_final, 6958, 5, models, lr, pca_model=pca)

# Applying the ranking_songs function
ranking_songs(hybrid_recommendations, final_play)

**Observations and Insights:**

- We use PCA and Linear Regression to build a hybrid model based on the 4 optimized models.
- The Hybrid model improve the performance greatly! The recall score increased to **0.819**! And F1 scores have been increased too. Though Precision score decreased a bit, and RMSE increased a bit.
- We implement the prediction and recommendation function for the hybrid model.

#### Hybrid - Ridge Regression recommendation system

In [ ]:
# Get predictions from models for the training set (recover from backup)
df_trainset_pred = df_trainset_pred_init

# Get predictions from models for the test set (recover from backup)
df_testset_pred = df_testset_pred_init

# Train a Ridge regression model using the PCA components
ridge = Ridge()
ridge.fit(X_train, y_train)

df_trainset_pred = hybrid_process(X_train, df_trainset_pred, ridge)

# Predict play count using the hybrid model
print('---------------------------------------------------------------------------------------------------------')
print('Apply on Test Set:')
df_testset_pred = hybrid_process(X_test, df_testset_pred, ridge)

In [ ]:
# Calc precision@k, recall@k, and F_1 score
precision, recall, f1, rmse = hybrid_precision_recall_at_k(df_testset_pred, k=30, threshold=1.5)

# Save the model's info
Hybrid_params = ridge.get_params()
add_model_record(df_model_info, 'Hybrid - Ridge Regression', Hybrid_params, precision, recall, f1, rmse, model_combination)

Not much difference with the Linear Regression model. Let's try to tune it.

#### Improving Hybrid - Ridge Regression recommendation system by tuning its hyper-parameters

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------------------
# Tune the Ridge Regression
#------------------------------------------------------------------------------------------------------------------------------------------
# Get predictions from models for the training set (recover from backup)
df_trainset_pred = df_trainset_pred_init

# Get predictions from models for the test set (recover from backup)
df_testset_pred = df_testset_pred_init

# Define Ridge regression model
ridge_optimized = Ridge()

# Define hyperparameters to tune
param_grid_ridge = {
    'alpha': [0.1, 1.0, 10.0, 100.0],
    'fit_intercept': [True, False],
    'max_iter': [None, 1000, 2000],
    'tol': [0.001, 0.0001],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
}

# Perform grid search with cross-validation
grid_search_ridge = SklearnGridSearchCV(ridge_optimized, param_grid_ridge, cv=5, scoring='neg_mean_squared_error')
grid_search_ridge.fit(X_train, y_train)

# Best estimator
print("Ridge regression - Best estimator :")
print(grid_search_ridge.best_estimator_)

# Best parameters
print("Ridge regression - Best parameters:")
print(grid_search_ridge.best_params_)

# Best score
print("Ridge regression - Best score     :")
print(grid_search_ridge.best_score_)

ridge_optimized = grid_search_ridge.best_estimator_

df_trainset_pred = hybrid_process(X_train, df_trainset_pred, ridge_optimized)

# Predict play count using the hybrid model
print('---------------------------------------------------------------------------------------------------------')
print('Apply on Test Set:')
df_testset_pred = hybrid_process(X_test, df_testset_pred, ridge_optimized)

In [ ]:
# Calc precision@k, recall@k, and F_1 score
precision, recall, f1, rmse = hybrid_precision_recall_at_k(df_testset_pred, k=30, threshold=1.5)

# Save the model's info
#Hybrid_params = grid_search_ridge.best_params_
Hybrid_params = str({**grid_search_ridge.best_params_,**ridge_optimized.get_params()})
add_model_record(df_model_info, 'Hybrid - Ridge Regression Optimized', Hybrid_params, precision, recall, f1, rmse, model_combination)

In [ ]:
# Check predictions for a user with an interacted song
h_pred = hybrid_predict(models, 6958, 1671, ridge_optimized)

It looks pretty good. The Ridge model predicted 1.8~ is very close the actual 2.0 play count.

In [ ]:
# Check predictions for a user with an not-interacted song
h_pred = hybrid_predict(models, 6958, 3232, ridge_optimized)

In [ ]:
# Making top 5 recommendations for a user_id with hybrid recommendation engine
hybrid_recommendations = hybrid_get_recommendations(df_final, 6958, 5, models, ridge_optimized)

# Applying the ranking_songs function
ranking_songs(hybrid_recommendations, final_play)

**Observations and Insights:**

- The Ridge regression models return almost the same scores, not much difference with the linear regression ones. The Optimized one got a little bit higher F1 score and Precision score, and the lowest RMSE.


#### Hybrid - Lasso Regression recommendation system

In [ ]:
# Get predictions from models for the training set (recover from backup)
df_trainset_pred = df_trainset_pred_init

# Get predictions from models for the test set (recover from backup)
df_testset_pred = df_testset_pred_init

# Train a Lasso regression model using the PCA components
lasso = Lasso()
lasso.fit(X_train, y_train)

df_trainset_pred = hybrid_process(X_train, df_trainset_pred, lasso)

# Predict play count using the hybrid model
print('---------------------------------------------------------------------------------------------------------')
print('Apply on Test Set:')
df_testset_pred = hybrid_process(X_test, df_testset_pred, lasso)



In [ ]:
# Calc precision@k, recall@k, and F_1 score
precision, recall, f1, rmse = hybrid_precision_recall_at_k(df_testset_pred, k=30, threshold=1.5)

There is something werid in the Lasso regression. Let's check any Constant Predictor exist.

In [ ]:
# check if the column has the same value for all rows
constant_columns = [col for col in df_trainset_pred.columns if df_trainset_pred[col].nunique() == 1]
print("Constant columns:", constant_columns)

In [ ]:
df_trainset_pred

In [ ]:
df_testset_pred

In [ ]:
# Check the actual play count's mean
print(df_trainset_pred['actual'].mean())
print(df_testset_pred['actual'].mean())

Yes, the Lasso regression model is **overshrinking**, driving all feature coefficients to zero, it only predicts the mean of the target variable, becoming a constant predictor.

Let's try to tune it.

#### Improving Hybrid - Lasso Regression recommendation system by tuning its hyper-parameters

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------------------
# Tune the Lasso Regression
#------------------------------------------------------------------------------------------------------------------------------------------
# Get predictions from models for the training set (recover from backup)
df_trainset_pred = df_trainset_pred_init

# Get predictions from models for the test set (recover from backup)
df_testset_pred = df_testset_pred_init

# Define lasso regression model
lasso_optimized = Lasso()

# Define hyperparameters to tune
param_grid_lasso = {
  'alpha': [0.01, 0.1, 1.0, 10.0, 100.0],    # Regularization strength for Lasso
  'max_iter': [1000, 2000, 3000],            # Maximum number of iterations
  'tol': [0.0001, 0.001, 0.01]               # Tolerance for stopping criteria
}

# Perform grid search with cross-validation
grid_search_lasso = SklearnGridSearchCV(lasso_optimized, param_grid_lasso, cv=5, scoring='neg_mean_squared_error')
grid_search_lasso.fit(X_train, y_train)

# Best estimator
print("Lasso regression - Best estimator :")
print(grid_search_lasso.best_estimator_)

# Best parameters
print("Lasso regression - Best parameters:")
print(grid_search_lasso.best_params_)

# Best score
print("Lasso regression - Best score     :")
print(grid_search_lasso.best_score_)

lasso_optimized = grid_search_lasso.best_estimator_

df_trainset_pred = hybrid_process(X_train, df_trainset_pred, lasso_optimized)

# Predict play count using the hybrid model
print('---------------------------------------------------------------------------------------------------------')
print('Apply on Test Set:')
df_testset_pred = hybrid_process(X_test, df_testset_pred, lasso_optimized)

In [ ]:
# Calc precision@k, recall@k, and F_1 score
precision, recall, f1, rmse = hybrid_precision_recall_at_k(df_testset_pred, k=30, threshold=1.5)

# Save the model's info
#Hybrid_params = grid_search_lasso.best_params_
Hybrid_params = str({**grid_search_lasso.best_params_,**lasso_optimized.get_params()})
add_model_record(df_model_info, 'Hybrid - Lasso Regression Optimized', Hybrid_params, precision, recall, f1, rmse, model_combination)

In [ ]:
# check if the column has the same value for all rows
constant_columns = [col for col in df_trainset_pred.columns if df_trainset_pred[col].nunique() == 1]
print("Constant columns:", constant_columns)

The tuning solved the shrinking problem. Let's try the prediction and recommendation function too.

In [ ]:
# Check predictions for a user with an interacted song
h_pred = hybrid_predict(models, 6958, 1671, lasso_optimized)

It looks pretty good. The Lasso model predicted 1.7~ is very close the actual 2.0 play count.

In [ ]:
# Check predictions for a user with an not-interacted song
h_pred = hybrid_predict(models, 6958, 3232, lasso_optimized)

In [ ]:
# Making top 5 recommendations for a user_id with hybrid recommendation engine
hybrid_recommendations = hybrid_get_recommendations(df_final, 6958, 5, models, lasso_optimized)

# Applying the ranking_songs function
ranking_songs(hybrid_recommendations, final_play)

**Observations and Insights:**

- Implement Linear, Ridge and Lasso regression to combine the 4 optimized models to build hybrid recommendation systems and also tune them.

- Evaluation method is included so that we can compare all models, especially with the 4 basic models.

- All the hybrid models have much higher recall scores than the stand-alone models.

- Met the overshrinking problem in Lasso models. But after tuned, the problem has been solved.

- So far we have 4 usable hybrid models, their performance are quite similar, all have **high recall score of ~0.81 and F1 score of ~0.53**, which are **higher** than all the stand-alone models. Though the precision score is lower than the stand-alone models except for Item-Item optimized.

- **The Tuned Lasso Regression model** got a a bit lower Recall score than the others, but the precision scores are a bit higher, that makes it got a bit higher F1 score. And RMSE is the lowest. It's the **best-performing model** so far.





#### Hybrid - combine SVD and User-User only

As SVD and User-User model has higher precision score, let's  combine these two to to see if there is any improvement.

In [ ]:
# Combine 2 models only
models = [svd_optimized, sim_user_user_optimized]
model_combination = 'SVD, U-U'

X_train, y_train, df_trainset_pred = hybrid_extract(models, trainset_tuples)
# Create a deep copy to backup the init. value
df_trainset_pred_init = df_trainset_pred.copy(deep=True)

# Get predictions from models for the test set
X_test, y_test, df_testset_pred = hybrid_extract(models, testset)
# Create a deep copy to backup the init. value
df_testset_pred_init = df_testset_pred.copy(deep=True)

In [ ]:
# Handle correlation with PCA
pca = PCA(n_components=len(models))
X_train_pca = pca.fit_transform(X_train)
print("Explained variance ratio:", pca.explained_variance_ratio_)
print("Total explained variance:", np.sum(pca.explained_variance_ratio_))
X_test_pca = pca.transform(X_test)

In [ ]:
# Get predictions from models for the training set (recover from backup)
df_trainset_pred = df_trainset_pred_init

# Get predictions from models for the test set (recover from backup)
df_testset_pred = df_testset_pred_init

# Train a linear regression model using the PCA components
lr = LinearRegression()
lr.fit(X_train_pca, y_train)

df_trainset_pred = hybrid_process(X_train_pca, df_trainset_pred, lr)

# Predict play count using the hybrid model
print('---------------------------------------------------------------------------------------------------------')
print('Apply on Test Set:')
df_testset_pred = hybrid_process(X_test_pca, df_testset_pred, lr)

In [ ]:
# Calc precision@k, recall@k, and F_1 score
precision, recall, f1, rmse = hybrid_precision_recall_at_k(df_testset_pred, k=30, threshold=1.5)

# Save the model's info
Hybrid_params = lr.get_params()
add_model_record(df_model_info, 'Hybrid - PCA and Linear Regression', Hybrid_params, precision, recall, f1, rmse, model_combination)

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------------------
# Tune the Ridge Regression
#------------------------------------------------------------------------------------------------------------------------------------------
# Get predictions from models for the training set (recover from backup)
df_trainset_pred = df_trainset_pred_init

# Get predictions from models for the test set (recover from backup)
df_testset_pred = df_testset_pred_init

# Define Ridge regression model
ridge_optimized = Ridge()

# Define hyperparameters to tune
param_grid_ridge = {
    'alpha': [0.1, 1.0, 10.0, 100.0],
    'fit_intercept': [True, False],
    'max_iter': [None, 1000, 2000],
    'tol': [0.001, 0.0001],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
}

# Perform grid search with cross-validation
grid_search_ridge = SklearnGridSearchCV(ridge_optimized, param_grid_ridge, cv=5, scoring='neg_mean_squared_error')
grid_search_ridge.fit(X_train, y_train)

# Best estimator
print("Ridge regression - Best estimator :")
print(grid_search_ridge.best_estimator_)

# Best parameters
print("Ridge regression - Best parameters:")
print(grid_search_ridge.best_params_)

# Best score
print("Ridge regression - Best score     :")
print(grid_search_ridge.best_score_)

ridge_optimized = grid_search_ridge.best_estimator_

df_trainset_pred = hybrid_process(X_train, df_trainset_pred, ridge_optimized)

# Predict play count using the hybrid model
print('---------------------------------------------------------------------------------------------------------')
print('Apply on Test Set:')
df_testset_pred = hybrid_process(X_test, df_testset_pred, ridge_optimized)

In [ ]:
# Calc precision@k, recall@k, and F_1 score
precision, recall, f1, rmse = hybrid_precision_recall_at_k(df_testset_pred, k=30, threshold=1.5)

# Save the model's info
#Hybrid_params = grid_search_ridge.best_params_
Hybrid_params = str({**grid_search_ridge.best_params_,**ridge_optimized.get_params()})
add_model_record(df_model_info, 'Hybrid - Ridge Regression Optimized', Hybrid_params, precision, recall, f1, rmse, model_combination)

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------------------
# Tune the Lasso Regression
#------------------------------------------------------------------------------------------------------------------------------------------
# Get predictions from models for the training set (recover from backup)
df_trainset_pred = df_trainset_pred_init

# Get predictions from models for the test set (recover from backup)
df_testset_pred = df_testset_pred_init

# Define lasso regression model
lasso_optimized = Lasso()

# Define hyperparameters to tune
param_grid_lasso = {
  'alpha': [0.01, 0.1, 1.0, 10.0, 100.0],    # Regularization strength for Lasso
  'max_iter': [1000, 2000, 3000],            # Maximum number of iterations
  'tol': [0.0001, 0.001, 0.01]               # Tolerance for stopping criteria
}

# Perform grid search with cross-validation
grid_search_lasso = SklearnGridSearchCV(lasso_optimized, param_grid_lasso, cv=5, scoring='neg_mean_squared_error')
grid_search_lasso.fit(X_train, y_train)

# Best estimator
print("Lasso regression - Best estimator :")
print(grid_search_lasso.best_estimator_)

# Best parameters
print("Lasso regression - Best parameters:")
print(grid_search_lasso.best_params_)

# Best score
print("Lasso regression - Best score     :")
print(grid_search_lasso.best_score_)

lasso_optimized = grid_search_lasso.best_estimator_

df_trainset_pred = hybrid_process(X_train, df_trainset_pred, lasso_optimized)

# Predict play count using the hybrid model
print('---------------------------------------------------------------------------------------------------------')
print('Apply on Test Set:')
df_testset_pred = hybrid_process(X_test, df_testset_pred, lasso_optimized)

In [ ]:
# Calc precision@k, recall@k, and F_1 score
precision, recall, f1, rmse = hybrid_precision_recall_at_k(df_testset_pred, k=30, threshold=1.5)

# Save the model's info
#Hybrid_params = grid_search_lasso.best_params_
Hybrid_params = str({**grid_search_lasso.best_params_,**lasso_optimized.get_params()})
add_model_record(df_model_info, 'Hybrid - Lasso Regression Optimized', Hybrid_params, precision, recall, f1, rmse, model_combination)

In [ ]:
# check if the column has the same value for all rows
constant_columns = [col for col in df_trainset_pred.columns if df_trainset_pred[col].nunique() == 1]
print("Constant columns:", constant_columns)

In [ ]:
# Check predictions for a user with an interacted song
h_pred = hybrid_predict(models, 6958, 1671, lasso_optimized)

It looks pretty good. The Lasso model predicted 1.7~ is very close the actual 2.0 play count.

In [ ]:
# Check predictions for a user with an not-interacted song
h_pred = hybrid_predict(models, 6958, 3232, lasso_optimized)

In [ ]:
# Making top 5 recommendations for a user_id with hybrid recommendation engine
hybrid_recommendations = hybrid_get_recommendations(df_final, 6958, 5, models, lasso_optimized)

# Applying the ranking_songs function
ranking_songs(hybrid_recommendations, final_play)

**Observations and Insights:**

- The precision score has **increased to ~0.397**. The F1 score has increased to **~0.53**.

#### Hybrid - combine User-User and Item-item only

As User-User and Item-item model complement each other, let's  combine these two to see if there is any improvement.

In [ ]:
# Combine 2 models only
models = [sim_user_user_optimized, sim_item_item_optimized]
model_combination = 'U-U, I-I'

X_train, y_train, df_trainset_pred = hybrid_extract(models, trainset_tuples)
# Create a deep copy to backup the init. value
df_trainset_pred_init = df_trainset_pred.copy(deep=True)

# Get predictions from models for the test set
X_test, y_test, df_testset_pred = hybrid_extract(models, testset)
# Create a deep copy to backup the init. value
df_testset_pred_init = df_testset_pred.copy(deep=True)

In [ ]:
# Handle correlation with PCA
pca = PCA(n_components=len(models))
X_train_pca = pca.fit_transform(X_train)
print("Explained variance ratio:", pca.explained_variance_ratio_)
print("Total explained variance:", np.sum(pca.explained_variance_ratio_))
X_test_pca = pca.transform(X_test)

In [ ]:
# Get predictions from models for the training set (recover from backup)
df_trainset_pred = df_trainset_pred_init

# Get predictions from models for the test set (recover from backup)
df_testset_pred = df_testset_pred_init

# Train a linear regression model using the PCA components
lr = LinearRegression()
lr.fit(X_train_pca, y_train)

df_trainset_pred = hybrid_process(X_train_pca, df_trainset_pred, lr)

# Predict play count using the hybrid model
print('---------------------------------------------------------------------------------------------------------')
print('Apply on Test Set:')
df_testset_pred = hybrid_process(X_test_pca, df_testset_pred, lr)

In [ ]:
# Calc precision@k, recall@k, and F_1 score
precision, recall, f1, rmse = hybrid_precision_recall_at_k(df_testset_pred, k=30, threshold=1.5)

# Save the model's info
Hybrid_params = lr.get_params()
add_model_record(df_model_info, 'Hybrid - PCA and Linear Regression', Hybrid_params, precision, recall, f1, rmse, model_combination)

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------------------
# Tune the Ridge Regression
#------------------------------------------------------------------------------------------------------------------------------------------
# Get predictions from models for the training set (recover from backup)
df_trainset_pred = df_trainset_pred_init

# Get predictions from models for the test set (recover from backup)
df_testset_pred = df_testset_pred_init

# Define Ridge regression model
ridge_optimized = Ridge()

# Define hyperparameters to tune
param_grid_ridge = {
    'alpha': [0.1, 1.0, 10.0, 100.0],
    'fit_intercept': [True, False],
    'max_iter': [None, 1000, 2000],
    'tol': [0.001, 0.0001],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
}

# Perform grid search with cross-validation
grid_search_ridge = SklearnGridSearchCV(ridge_optimized, param_grid_ridge, cv=5, scoring='neg_mean_squared_error')
grid_search_ridge.fit(X_train, y_train)

# Best estimator
print("Ridge regression - Best estimator :")
print(grid_search_ridge.best_estimator_)

# Best parameters
print("Ridge regression - Best parameters:")
print(grid_search_ridge.best_params_)

# Best score
print("Ridge regression - Best score     :")
print(grid_search_ridge.best_score_)

ridge_optimized = grid_search_ridge.best_estimator_

df_trainset_pred = hybrid_process(X_train, df_trainset_pred, ridge_optimized)

# Predict play count using the hybrid model
print('---------------------------------------------------------------------------------------------------------')
print('Apply on Test Set:')
df_testset_pred = hybrid_process(X_test, df_testset_pred, ridge_optimized)

In [ ]:
# Calc precision@k, recall@k, and F_1 score
precision, recall, f1, rmse = hybrid_precision_recall_at_k(df_testset_pred, k=30, threshold=1.5)

# Save the model's info
#Hybrid_params = grid_search_ridge.best_params_
Hybrid_params = str({**grid_search_ridge.best_params_,**ridge_optimized.get_params()})
add_model_record(df_model_info, 'Hybrid - Ridge Regression Optimized', Hybrid_params, precision, recall, f1, rmse, model_combination)

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------------------
# Tune the Lasso Regression
#------------------------------------------------------------------------------------------------------------------------------------------
# Get predictions from models for the training set (recover from backup)
df_trainset_pred = df_trainset_pred_init

# Get predictions from models for the test set (recover from backup)
df_testset_pred = df_testset_pred_init

# Define lasso regression model
lasso_optimized = Lasso()

# Define hyperparameters to tune
param_grid_lasso = {
  'alpha': [0.01, 0.1, 1.0, 10.0, 100.0],    # Regularization strength for Lasso
  'max_iter': [1000, 2000, 3000],            # Maximum number of iterations
  'tol': [0.0001, 0.001, 0.01]               # Tolerance for stopping criteria
}

# Perform grid search with cross-validation
grid_search_lasso = SklearnGridSearchCV(lasso_optimized, param_grid_lasso, cv=5, scoring='neg_mean_squared_error')
grid_search_lasso.fit(X_train, y_train)

# Best estimator
print("Lasso regression - Best estimator :")
print(grid_search_lasso.best_estimator_)

# Best parameters
print("Lasso regression - Best parameters:")
print(grid_search_lasso.best_params_)

# Best score
print("Lasso regression - Best score     :")
print(grid_search_lasso.best_score_)

lasso_optimized = grid_search_lasso.best_estimator_

df_trainset_pred = hybrid_process(X_train, df_trainset_pred, lasso_optimized)

# Predict play count using the hybrid model
print('---------------------------------------------------------------------------------------------------------')
print('Apply on Test Set:')
df_testset_pred = hybrid_process(X_test, df_testset_pred, lasso_optimized)

In [ ]:
# Calc precision@k, recall@k, and F_1 score
precision, recall, f1, rmse = hybrid_precision_recall_at_k(df_testset_pred, k=30, threshold=1.5)

# Save the model's info
#Hybrid_params = grid_search_lasso.best_params_
Hybrid_params = str({**grid_search_lasso.best_params_,**lasso_optimized.get_params()})
add_model_record(df_model_info, 'Hybrid - Lasso Regression Optimized', Hybrid_params, precision, recall, f1, rmse, model_combination)

In [ ]:
# check if the column has the same value for all rows
constant_columns = [col for col in df_trainset_pred.columns if df_trainset_pred[col].nunique() == 1]
print("Constant columns:", constant_columns)

In [ ]:
# Check predictions for a user with an interacted song
h_pred = hybrid_predict(models, 6958, 1671, lasso_optimized)

It looks pretty good. The Lasso model predicted 1.7~ is very close the actual 2.0 play count.

In [ ]:
# Check predictions for a user with an not-interacted song
h_pred = hybrid_predict(models, 6958, 3232, lasso_optimized)

In [ ]:
# Making top 5 recommendations for a user_id with hybrid recommendation engine
hybrid_recommendations = hybrid_get_recommendations(df_final, 6958, 5, models, lasso_optimized)

# Applying the ranking_songs function
ranking_songs(hybrid_recommendations, final_play)

**Observations and Insights:**

- The precision score has **increased to ~0.400**. F1 score is **~0.53**.

#### Hybrid - Random Forest recommendation system

Let's build a recommendation system based on four basic optimized models. The main idea is to prioritize songs that are recommended more frequently, giving them a higher rank.

In [ ]:
def get_recommendations_forest(data, final_play, user_id, top_n, models):

    # Initialize an empty list to hold all DataFrames
    all_ranked_songs = []

    for model_index, model in enumerate(models):
        recommendations = get_recommendations(data, user_id, top_n, model)
        ranked_songs = ranking_songs(recommendations, final_play)

        #df_ranked_songs = pd.DataFrame(ranked_songs)
        # Add the model and user_id information
        ranked_songs['model'] = f'model_{model_index + 1}'
        ranked_songs['uid'] = user_id

        # Append the DataFrame to the list
        all_ranked_songs.append(ranked_songs)

    # Concatenate all DataFrames in the list into a single DataFrame
    df_ranked_songs = pd.concat(all_ranked_songs, ignore_index=True)

    # reset the index
    df_ranked_songs.reset_index(drop=True, inplace=True)

    # Group by 'song_id' and calculate the mean of 'play_count' and value counts
    df_ranked_songs_count = df_ranked_songs.groupby('song_id').agg(
        mean_play_count=('corrected_play_count', 'mean'),
        value_count=('song_id', 'size')
    ).reset_index()

    # Sort by 'value_count' in descending order, mean_play_count in descending order
    df_ranked_songs_count_sorted = df_ranked_songs_count.sort_values(by=['value_count', 'mean_play_count'], ascending=[False, False]).reset_index()

    return df_ranked_songs_count_sorted[:top_n] # Returing top n songs for this user

In [ ]:
models = [svd_optimized, sim_user_user_optimized, sim_item_item_optimized, clust_tuned]
forest_recommendations = get_recommendations_forest(df_final, final_play, 6958, 5, models)
print(forest_recommendations)

**Observations and Insights:**



*   We implement a function to Make top n recommendations for any user id based on four basic optimized models. The final ranking will first be sorted by the number of recommendations in descending order and then by the average play count prediction in descending order. It will return the song ids and aver. predicted play counts.
*   The function inherently uses the corrected play count, applying fewer penalties to frequently played songs and more penalties to less frequently played songs.



#### Comparing all the models

In [ ]:
# Check the model info dataframe
df_model_info

In [ ]:
# Combine 'Model Type' and 'Combination' into a new 'Display Model Type' column
df_model_info['Display Model Type'] = df_model_info['Model Type'] + " - " + df_model_info['Combination']

# Set the 'Display Model Type' column as the index for plotting
df_model_info.set_index('Display Model Type', inplace=True)

# Plot the line plot for each metric in the same chart
plt.figure(figsize=(14, 12))

plt.plot(df_model_info.index, df_model_info['Precision'], marker='o', label='Precision')
plt.plot(df_model_info.index, df_model_info['Recall'], marker='s', label='Recall')
plt.plot(df_model_info.index, df_model_info['F1'], marker='^', label='F1')
plt.plot(df_model_info.index, df_model_info['RMSE'], marker='d', label='RMSE')

plt.title('Performance Metrics by Model Type')
plt.xlabel('Model Type')
plt.ylabel('Scores')
plt.xticks(rotation=90)
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## **Conclusion and Recommendations**


We build various recommendation systems:
- Popularity-Based
- Similarity-Based Collaborative filtering (user-user and item-item)
- Matrix Factorization Based Collaborative Filtering
- Clustering-based
- Content-based collaborative filtering
- Hybrid (combine SVD, user-user, item-item and Clustering-based, with Linear, Ridge and Lasso regression)
- Hybrid - Random Forest

**Insights:**

- Our dataset contains limited information about the songs and users themselves, necessitating a heavy reliance on user-song interactions to uncover latent features and clustering patterns.

- We face the challenge of data sparsity which is common in recommendation systems.

- Among these techniques, hybrid models generally outperform others in sparse data scenarios by integrating various data sources and methodologies, and leveraging the complementary strengths of different approaches.

- Matrix Factorization models also show strong performance due to their ability to uncover latent factors and generalize well with limited data.

- Content-based and simple collaborative filtering methods may lag behind in sparse datasets due to their reliance on either detailed metadata or sufficient user-item interactions.

**Proposal for the final solution design:**

*   **Hybrid model which combines SVD and user-user with Ridge regression**

*   It has the **highest F1 score (0.530484)** and **high recall score (0.798430)** among all the models we tried, precision score is close the other models', making it the **best-performing model**.

*   High recall can lead to more engagement, as users find more songs that they enjoy. This is particularly important in competitive markets where retaining user attention is crucial.

*   *Hybrid model which combines SVD, user-user, item-item and Clustering-based model with Lasso regression* maybe the *2nd choice*. As It has the **highest recall score (0.819106)** and **high F1 score (0.525687)** among all the models we tried.
   



**Scope to improve:**

- Enhancing Data Quality and Quantity: Collecting more interaction data and enriching item and user metadata can significantly improve model performance.

- Parameter Tuning and Optimization: Fine-tuning hyperparameters and optimizing model architecture can yield better results.

- Advanced Techniques: Incorporating advanced machine learning techniques, such as deep learning (e.g., neural collaborative filtering, autoencoders) and graph-based methods (e.g., Graph Convolutional Networks), can enhance the model's ability to capture complex patterns.

- User Feedback Integration: Continuously incorporating user feedback and engagement data can refine recommendations and adapt to changing user preferences.

- Regular Updates and Retraining: Regularly updating and retraining models with new data helps maintain accuracy and relevance over time.